In [1]:
import pandas as pd
import numpy as np

In [2]:
schools_complete_csv_path = "raw_data/schools_complete.csv"
students_complete_csv_path = "raw_data/students_complete.csv"

In [3]:
schools_complete_df = pd.read_csv(schools_complete_csv_path)
students_complete_df = pd.read_csv(students_complete_csv_path)

In [4]:
#schools_complete_df

In [5]:
#students_complete_df.head()

In [6]:
total_budget = schools_complete_df['budget'].sum()

In [7]:
total_schools = schools_complete_df['name'].count()

In [8]:
total_students = students_complete_df['name'].count()

In [9]:
average_math_score = students_complete_df['math_score'].mean()
average_reading_score = students_complete_df['reading_score'].mean()


In [10]:
bins = [0, 70, 100]
group_names = ['Failed', 'Passed']
students_complete_df['reading_grade_status'] = pd.cut(students_complete_df["reading_score"], bins, labels=group_names)
students_complete_df['math_grade_status'] = pd.cut(students_complete_df["math_score"], bins, labels=group_names)
#students_complete_df['math_grade_status'].value_counts()['Passed']
percent_passing_math = ((students_complete_df['math_grade_status'].value_counts()['Passed'])/total_students) * 100
percent_passing_reading = ((students_complete_df['reading_grade_status'].value_counts()['Passed'])/total_students) * 100


In [11]:
students_complete_df['overall_average_score'] = (students_complete_df['reading_score'] + students_complete_df['math_score'])/2
students_complete_df = students_complete_df[['Student ID', 'name', 'gender', 'grade', 'school', 'reading_score', 'reading_grade_status',
       'math_score', 'math_grade_status','overall_average_score']]
overall_passing_rate = (percent_passing_math + percent_passing_reading)/2

In [12]:
#students_complete_df.head()
#students_complete_df["math_grade_status"].value_counts()

In [13]:
district_summary_table = pd.DataFrame({"Total Schools":[total_schools],"Total Students":[total_students],
                                       "Total Budget":[total_budget],"Average Math Score":[average_math_score],
                                       "Average Reading Score":[average_reading_score],"% Passing Math":[percent_passing_math],
                                       "% Passing Reading":[percent_passing_reading],"Overall Passing Rate":[overall_passing_rate]})
district_summary_table = district_summary_table[["Total Schools","Total Students","Total Budget","Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","Overall Passing Rate"]]


In [14]:
district_summary_table


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,39170,24649428,78.985371,81.87784,72.392137,82.971662,77.681899


In [15]:
schools_complete_df = schools_complete_df.rename(columns={"name":"school"})
#schools_complete_df

In [16]:
merge_schools_student_table_df = pd.merge(schools_complete_df, students_complete_df, on="school", how="left")
#merge_schools_student_table_df.head()

In [17]:
total_students_count_per_school = merge_schools_student_table_df["school"].value_counts()
school_summary_total_students_df = pd.DataFrame(total_students_count_per_school)
school_summary_total_students_df.reset_index(inplace=True)
school_summary_total_students_df.columns = ["School Name", "Total Students"]

In [18]:
#school_summary_total_students_df

In [19]:
schools_complete_school_rename_df = schools_complete_df.rename(columns={"school":"School Name"})
school_summary_merge1_df = pd.merge(schools_complete_school_rename_df, school_summary_total_students_df, on="School Name")
school_summary_merge1_df = school_summary_merge1_df.rename(columns={"budget":"Total School Budget", "type":"School Type"})
school_summary_merge1_df["Per Student Budget"] = (school_summary_merge1_df ["Total School Budget"] / school_summary_merge1_df ["Total Students"])
school_summary_merge1_df = school_summary_merge1_df[["School Name","School Type","Total Students","Total School Budget","Per Student Budget"]]
#school_summary_merge1_df

In [20]:
grouped_by_school_total_rading_math_score_df = pd.DataFrame(merge_schools_student_table_df.groupby("school")["reading_score","math_score"].sum())
grouped_by_school_total_rading_math_score_df.reset_index(inplace=True)
grouped_by_school_total_rading_math_score_df.columns=["school", "reading_score","math_score"]
grouped_by_school_total_rading_math_score_df = grouped_by_school_total_rading_math_score_df.rename(columns={"school":"School Name"})
#grouped_by_school_total_rading_math_score_df
school_student_summary_per_school_merge2_df = pd.merge(school_summary_total_students_df, grouped_by_school_total_rading_math_score_df, on="School Name")
school_student_summary_per_school_merge2_df
school_student_summary_per_school_merge2_df ["Average Math Score"] = school_student_summary_per_school_merge2_df ["math_score"] / school_student_summary_per_school_merge2_df["Total Students"]
school_student_summary_per_school_merge2_df ["Average Reading Score"] = school_student_summary_per_school_merge2_df ["reading_score"] / school_student_summary_per_school_merge2_df["Total Students"]
school_student_summary_per_school_merge2_df
school_student_summary_per_school_merge2_df = school_student_summary_per_school_merge2_df[["School Name", "Average Math Score","Average Reading Score"]]
#school_student_summary_per_school_merge2_df

In [21]:
school_summary_merge3_df = pd.merge(school_summary_merge1_df, school_student_summary_per_school_merge2_df, on="School Name")
#school_summary_merge3_df

In [22]:
students_complete_reading_passed_df = students_complete_df.loc[students_complete_df["reading_grade_status"] == "Passed",:]
students_complete_reading_passed_df = students_complete_reading_passed_df [["school","reading_grade_status"]]
students_complete_reading_passed_per_school_df = students_complete_reading_passed_df["school"].value_counts()
students_complete_reading_passed_per_school_df = pd.DataFrame(students_complete_reading_passed_per_school_df)
students_complete_reading_passed_per_school_df.reset_index(inplace=True)
students_complete_reading_passed_per_school_df.columns = ["School Name", "Total Passing Reading"]
#students_complete_reading_passed_per_school_df 
school_student_summary_per_school_merge4_df = pd.merge(school_summary_total_students_df, students_complete_reading_passed_per_school_df, on="School Name")
school_student_summary_per_school_merge4_df["% Passing Reading"] = (school_student_summary_per_school_merge4_df["Total Passing Reading"] / school_student_summary_per_school_merge4_df["Total Students"]) * 100
#school_student_summary_per_school_merge4_df
school_student_summary_per_school_merge4_df = school_student_summary_per_school_merge4_df [["School Name","% Passing Reading"]]
#school_student_summary_per_school_merge4_df

In [23]:
school_summary_merge5_df = pd.merge(school_summary_merge3_df, school_student_summary_per_school_merge4_df, on="School Name")
#school_summary_merge5_df

In [24]:
students_complete_math_passed_df = students_complete_df.loc[students_complete_df["math_grade_status"] == "Passed",:]
students_complete_math_passed_df = students_complete_math_passed_df [["school","math_grade_status"]]
students_complete_math_passed_per_school_df = students_complete_math_passed_df["school"].value_counts()
students_complete_math_passed_per_school_df = pd.DataFrame(students_complete_math_passed_per_school_df)
students_complete_math_passed_per_school_df.reset_index(inplace=True)
students_complete_math_passed_per_school_df.columns = ["School Name", "Total Passing Math"]
#students_complete_math_passed_per_school_df 
school_student_summary_per_school_merge6_df = pd.merge(school_summary_total_students_df, students_complete_math_passed_per_school_df, on="School Name")
school_student_summary_per_school_merge6_df["% Passing Math"] = (school_student_summary_per_school_merge6_df["Total Passing Math"] / school_student_summary_per_school_merge6_df["Total Students"]) * 100
#school_student_summary_per_school_merge6_df
school_student_summary_per_school_merge6_df = school_student_summary_per_school_merge6_df [["School Name","% Passing Math"]]
#school_student_summary_per_school_merge6_df

In [25]:
school_summary_merge7_df = pd.merge(school_summary_merge5_df, school_student_summary_per_school_merge6_df, on="School Name")
#school_summary_merge7_df

In [26]:
school_summary_merge7_df["Overall Passing Rate"] = (school_summary_merge7_df["% Passing Reading"] + school_summary_merge7_df["% Passing Math"]) / 2
#school_summary_merge7_df

In [27]:
final_school_summary_merge7_index_df = school_summary_merge7_df.set_index("School Name")
final_school_summary_merge7_index_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Reading,% Passing Math,Overall Passing Rate
School Name,,,,,,,,,
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,78.813850,63.318478,71.066164
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,78.433367,63.750424,71.091896
Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,92.617831,89.892107,91.254969
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,78.187702,64.746494,71.467098
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,89.713896,91.553134
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.254490,90.932983,92.093736
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,93.864370,89.558665,91.711518
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,79.300643,64.630225,71.965434
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.740047,90.632319,91.686183


In [28]:
#final_school_summary_merge7_index_df["Average Math Score"] = final_school_summary_merge7_index_df["Average Math Score"].map("{0:,.2f}%".format)
#final_school_summary_merge7_index_df["Average Reading Score"] = final_school_summary_merge7_index_df["Average Reading Score"].map("{0:,.2f}%".format)
#final_school_summary_merge7_index_df["% Passing Reading"] = final_school_summary_merge7_index_df["% Passing Reading"].map("{0:,.2f}%".format)
#final_school_summary_merge7_index_df["% Passing Math"] = final_school_summary_merge7_index_df["% Passing Math"].map("{0:,.2f}%".format)
#final_school_summary_merge7_index_df["Overall Passing Rate"] = final_school_summary_merge7_index_df["Overall Passing Rate"].map("{0:,.2f}%".format)
#final_school_summary_merge7_index_df

In [29]:
top_school_summary_merge7_index_df = final_school_summary_merge7_index_df.sort_values("Overall Passing Rate", ascending=False)
top_school_summary_merge7_index_df.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Reading,% Passing Math,Overall Passing Rate
School Name,,,,,,,,,
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.254490,90.932983,92.093736
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,92.203742,91.683992,91.943867
Wright High School,Charter,1800,1049400,583.0,83.682222,83.955000,93.444444,90.277778,91.861111
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,93.864370,89.558665,91.711518
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.740047,90.632319,91.686183


In [30]:
bottom_school_summary_merge7_index_df = final_school_summary_merge7_index_df.sort_values("Overall Passing Rate", ascending=True)
bottom_school_summary_merge7_index_df.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Reading,% Passing Math,Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,77.744436,64.066017,70.905226
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,78.813850,63.318478,71.066164
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,78.281874,63.852132,71.067003
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,78.433367,63.750424,71.091896
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,78.187702,64.746494,71.467098


In [31]:
math_scores_by_grade_merge_schools_student_table_df = merge_schools_student_table_df[["school","grade","math_score" ]]
math_scores_by_grade_merge_schools_student_table_df = math_scores_by_grade_merge_schools_student_table_df.rename(columns={"school":"School Name"})
#math_scores_by_grade_merge_schools_student_table_df["grade"].value_counts()


In [32]:
math_scores_by_grade_merge_schools_student_table_9th_df = math_scores_by_grade_merge_schools_student_table_df.loc[math_scores_by_grade_merge_schools_student_table_df["grade"] == "9th"]
#math_scores_by_grade_merge_schools_student_table_9th_df
grouped_by_math_scores_by_grade_merge_schools_student_table_9th_df = pd.DataFrame(math_scores_by_grade_merge_schools_student_table_9th_df.groupby("School Name")["math_score"].mean())
grouped_by_math_scores_by_grade_merge_schools_student_table_9th_df.reset_index(inplace=True)
grouped_by_math_scores_by_grade_merge_schools_student_table_9th_df =grouped_by_math_scores_by_grade_merge_schools_student_table_9th_df.rename(columns={"math_score":"9th"})
#grouped_by_math_scores_by_grade_merge_schools_student_table_9th_df


math_scores_by_grade_merge_schools_student_table_10th_df = math_scores_by_grade_merge_schools_student_table_df.loc[math_scores_by_grade_merge_schools_student_table_df["grade"] == "10th"]
#math_scores_by_grade_merge_schools_student_table_10th_df
grouped_by_math_scores_by_grade_merge_schools_student_table_10th_df = pd.DataFrame(math_scores_by_grade_merge_schools_student_table_10th_df.groupby("School Name")["math_score"].mean())
grouped_by_math_scores_by_grade_merge_schools_student_table_10th_df.reset_index(inplace=True)
grouped_by_math_scores_by_grade_merge_schools_student_table_10th_df =grouped_by_math_scores_by_grade_merge_schools_student_table_10th_df.rename(columns={"math_score":"10th"})
#grouped_by_math_scores_by_grade_merge_schools_student_table_10th_df

math_scores_by_grade_merge_schools_student_table_11th_df = math_scores_by_grade_merge_schools_student_table_df.loc[math_scores_by_grade_merge_schools_student_table_df["grade"] == "11th"]
#math_scores_by_grade_merge_schools_student_table_11th_df
grouped_by_math_scores_by_grade_merge_schools_student_table_11th_df = pd.DataFrame(math_scores_by_grade_merge_schools_student_table_11th_df.groupby("School Name")["math_score"].mean())
grouped_by_math_scores_by_grade_merge_schools_student_table_11th_df.reset_index(inplace=True)
grouped_by_math_scores_by_grade_merge_schools_student_table_11th_df =grouped_by_math_scores_by_grade_merge_schools_student_table_11th_df.rename(columns={"math_score":"11th"})
#grouped_by_math_scores_by_grade_merge_schools_student_table_11th_df


math_scores_by_grade_merge_schools_student_table_12th_df = math_scores_by_grade_merge_schools_student_table_df.loc[math_scores_by_grade_merge_schools_student_table_df["grade"] == "12th"]
#math_scores_by_grade_merge_schools_student_table_12th_df
grouped_by_math_scores_by_grade_merge_schools_student_table_12th_df = pd.DataFrame(math_scores_by_grade_merge_schools_student_table_12th_df.groupby("School Name")["math_score"].mean())
grouped_by_math_scores_by_grade_merge_schools_student_table_12th_df.reset_index(inplace=True)
grouped_by_math_scores_by_grade_merge_schools_student_table_12th_df =grouped_by_math_scores_by_grade_merge_schools_student_table_12th_df.rename(columns={"math_score":"12th"})
#grouped_by_math_scores_by_grade_merge_schools_student_table_12th_df


In [33]:
math_scores_by_grade_1st_half_df = pd.merge(grouped_by_math_scores_by_grade_merge_schools_student_table_9th_df, 
                                grouped_by_math_scores_by_grade_merge_schools_student_table_10th_df,  on="School Name")


math_scores_by_grade_2nd_half_df = pd.merge(grouped_by_math_scores_by_grade_merge_schools_student_table_11th_df,
                                grouped_by_math_scores_by_grade_merge_schools_student_table_12th_df, on="School Name")

math_scores_by_grade_df = pd.merge(math_scores_by_grade_1st_half_df,math_scores_by_grade_2nd_half_df, on="School Name")
math_scores_by_grade_df = math_scores_by_grade_df.set_index("School Name")
math_scores_by_grade_df

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [34]:
reading_scores_by_grade_merge_schools_student_table_df = merge_schools_student_table_df[["school","grade","reading_score" ]]
reading_scores_by_grade_merge_schools_student_table_df = reading_scores_by_grade_merge_schools_student_table_df.rename(columns={"school":"School Name"})
#reading_scores_by_grade_merge_schools_student_table_df


In [35]:
reading_scores_by_grade_merge_schools_student_table_9th_df = reading_scores_by_grade_merge_schools_student_table_df.loc[reading_scores_by_grade_merge_schools_student_table_df["grade"] == "9th"]
#reading_scores_by_grade_merge_schools_student_table_9th_df
grouped_by_reading_scores_by_grade_merge_schools_student_table_9th_df = pd.DataFrame(reading_scores_by_grade_merge_schools_student_table_9th_df.groupby("School Name")["reading_score"].mean())
grouped_by_reading_scores_by_grade_merge_schools_student_table_9th_df.reset_index(inplace=True)
grouped_by_reading_scores_by_grade_merge_schools_student_table_9th_df =grouped_by_reading_scores_by_grade_merge_schools_student_table_9th_df.rename(columns={"reading_score":"9th"})
#grouped_by_reading_scores_by_grade_merge_schools_student_table_9th_df


reading_scores_by_grade_merge_schools_student_table_10th_df = reading_scores_by_grade_merge_schools_student_table_df.loc[reading_scores_by_grade_merge_schools_student_table_df["grade"] == "10th"]
#reading_scores_by_grade_merge_schools_student_table_10th_df
grouped_by_reading_scores_by_grade_merge_schools_student_table_10th_df = pd.DataFrame(reading_scores_by_grade_merge_schools_student_table_10th_df.groupby("School Name")["reading_score"].mean())
grouped_by_reading_scores_by_grade_merge_schools_student_table_10th_df.reset_index(inplace=True)
grouped_by_reading_scores_by_grade_merge_schools_student_table_10th_df =grouped_by_reading_scores_by_grade_merge_schools_student_table_10th_df.rename(columns={"reading_score":"10th"})
#grouped_by_reading_scores_by_grade_merge_schools_student_table_10th_df

reading_scores_by_grade_merge_schools_student_table_11th_df = reading_scores_by_grade_merge_schools_student_table_df.loc[reading_scores_by_grade_merge_schools_student_table_df["grade"] == "11th"]
#reading_scores_by_grade_merge_schools_student_table_11th_df
grouped_by_reading_scores_by_grade_merge_schools_student_table_11th_df = pd.DataFrame(reading_scores_by_grade_merge_schools_student_table_11th_df.groupby("School Name")["reading_score"].mean())
grouped_by_reading_scores_by_grade_merge_schools_student_table_11th_df.reset_index(inplace=True)
grouped_by_reading_scores_by_grade_merge_schools_student_table_11th_df =grouped_by_reading_scores_by_grade_merge_schools_student_table_11th_df.rename(columns={"reading_score":"11th"})
#grouped_by_reading_scores_by_grade_merge_schools_student_table_11th_df


reading_scores_by_grade_merge_schools_student_table_12th_df = reading_scores_by_grade_merge_schools_student_table_df.loc[reading_scores_by_grade_merge_schools_student_table_df["grade"] == "12th"]
#reading_scores_by_grade_merge_schools_student_table_12th_df
grouped_by_reading_scores_by_grade_merge_schools_student_table_12th_df = pd.DataFrame(reading_scores_by_grade_merge_schools_student_table_12th_df.groupby("School Name")["reading_score"].mean())
grouped_by_reading_scores_by_grade_merge_schools_student_table_12th_df.reset_index(inplace=True)
grouped_by_reading_scores_by_grade_merge_schools_student_table_12th_df =grouped_by_reading_scores_by_grade_merge_schools_student_table_12th_df.rename(columns={"reading_score":"12th"})
#grouped_by_reading_scores_by_grade_merge_schools_student_table_12th_df

In [36]:
reading_scores_by_grade_1st_half_df = pd.merge(grouped_by_reading_scores_by_grade_merge_schools_student_table_9th_df, 
                                grouped_by_reading_scores_by_grade_merge_schools_student_table_10th_df,  on="School Name")


reading_scores_by_grade_2nd_half_df = pd.merge(grouped_by_reading_scores_by_grade_merge_schools_student_table_11th_df,
                                grouped_by_reading_scores_by_grade_merge_schools_student_table_12th_df, on="School Name")

reading_scores_by_grade_df = pd.merge(reading_scores_by_grade_1st_half_df,reading_scores_by_grade_2nd_half_df, on="School Name")
reading_scores_by_grade_df = reading_scores_by_grade_df.set_index("School Name")
reading_scores_by_grade_df

,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [37]:
grouped_by_school_total_rading_math_score_df
school_summary_merge1_df
students_complete_reading_passed_per_school_df
students_complete_math_passed_per_school_df
final_merger_for_per_student_spending_df = pd.merge(school_summary_merge1_df,grouped_by_school_total_rading_math_score_df,  on="School Name")
final_merger_for_per_student_spending_df = pd.merge(final_merger_for_per_student_spending_df,students_complete_reading_passed_per_school_df,  on="School Name")
final_merger_for_per_student_spending_df = pd.merge(final_merger_for_per_student_spending_df,students_complete_math_passed_per_school_df,  on="School Name")
final_merger_for_per_student_spending_df.columns

Index(['School Name', 'School Type', 'Total Students', 'Total School Budget',
       'Per Student Budget', 'reading_score', 'math_score',
       'Total Passing Reading', 'Total Passing Math'],
      dtype='object')

In [38]:
reduced_final_merger_for_per_student_spending_df = final_merger_for_per_student_spending_df[['Per Student Budget','School Type', 'Total Students', 'Total School Budget', 'reading_score', 'math_score','Total Passing Reading', 'Total Passing Math']]
#reduced_final_merger_for_per_student_spending_df

In [39]:
bins = [500, 585, 600, 625, 660]
group_names = ['<$585', '$585-$600', '$600-$625', '$625-$660']

In [40]:
reduced_final_merger_for_per_student_spending_df["Spending Ranges (Per Student)"] = pd.cut(reduced_final_merger_for_per_student_spending_df["Per Student Budget"], bins, labels=group_names)
#reduced_final_merger_for_per_student_spending_df


In [41]:
rearange_reduced_final_merger_for_per_student_spending_df = reduced_final_merger_for_per_student_spending_df [["Spending Ranges (Per Student)",'Per Student Budget', 'School Type', 'Total Students','Total School Budget', 'reading_score', 'math_score','Total Passing Reading', 'Total Passing Math',]]
#rearange_reduced_final_merger_for_per_student_spending_df

In [42]:
groupedby_rearange_reduced_final_merger_for_per_student_spending_df = pd.DataFrame(rearange_reduced_final_merger_for_per_student_spending_df.groupby("Spending Ranges (Per Student)")['Per Student Budget', 'School Type','Total Students', 'Total School Budget', 'reading_score', 'math_score','Total Passing Reading', 'Total Passing Math'].sum())
groupedby_rearange_reduced_final_merger_for_per_student_spending_df["Average Math Score"] = groupedby_rearange_reduced_final_merger_for_per_student_spending_df["math_score"] / groupedby_rearange_reduced_final_merger_for_per_student_spending_df["Total Students"]
groupedby_rearange_reduced_final_merger_for_per_student_spending_df["Average Reading Score"] = groupedby_rearange_reduced_final_merger_for_per_student_spending_df["reading_score"] / groupedby_rearange_reduced_final_merger_for_per_student_spending_df["Total Students"]
groupedby_rearange_reduced_final_merger_for_per_student_spending_df["% Passing Math"] = (groupedby_rearange_reduced_final_merger_for_per_student_spending_df ["Total Passing Math"] / groupedby_rearange_reduced_final_merger_for_per_student_spending_df["Total Students"]) * 100
groupedby_rearange_reduced_final_merger_for_per_student_spending_df["% Passing Reading"] = (groupedby_rearange_reduced_final_merger_for_per_student_spending_df ["Total Passing Reading"] / groupedby_rearange_reduced_final_merger_for_per_student_spending_df["Total Students"]) * 100
groupedby_rearange_reduced_final_merger_for_per_student_spending_df["% Overall Passing Rate"] = (groupedby_rearange_reduced_final_merger_for_per_student_spending_df ["% Passing Math"] + groupedby_rearange_reduced_final_merger_for_per_student_spending_df ["% Passing Reading"]) / 2
reduced_groupedby_rearange_reduced_final_merger_for_per_student_spending_df = groupedby_rearange_reduced_final_merger_for_per_student_spending_df[["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing Rate"]]
reduced_groupedby_rearange_reduced_final_merger_for_per_student_spending_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.363065,83.964039,90.326633,93.451633,91.889133
$585-$600,83.359455,83.725724,89.892107,92.617831,91.254969
$600-$625,83.544856,83.906996,90.493827,92.921811,91.707819
$625-$660,77.354549,81.127434,65.785887,79.200308,72.493097


In [43]:
reduced_final_merger_for_per_school_size_df = final_merger_for_per_student_spending_df[['Total Students','Per Student Budget','School Type','Total School Budget', 'reading_score', 'math_score','Total Passing Reading', 'Total Passing Math']]
#reduced_final_merger_for_per_school_size_df


In [44]:
bins = [100, 1000, 3000, 5000]
group_names = ['Small(<1000)', 'Medium(1000-3000)', 'Large(3000-5000)']
reduced_final_merger_for_per_school_size_df["School Size"] = pd.cut(reduced_final_merger_for_per_school_size_df["Total Students"], bins, labels=group_names)
#reduced_final_merger_for_per_school_size_df

In [45]:
rearange_reduced_final_merger_for_per_school_size_df = reduced_final_merger_for_per_school_size_df[['School Size','Total Students', 'Per Student Budget', 'School Type','Total School Budget', 'reading_score', 'math_score','Total Passing Reading', 'Total Passing Math']]
#rearange_reduced_final_merger_for_per_school_size_df

In [46]:
groupedby_rearange_reduced_final_merger_for_per_school_size_df = pd.DataFrame(rearange_reduced_final_merger_for_per_school_size_df.groupby("School Size")['Per Student Budget', 'School Type','Total Students', 'Total School Budget', 'reading_score', 'math_score','Total Passing Reading', 'Total Passing Math'].sum())
groupedby_rearange_reduced_final_merger_for_per_school_size_df["Average Math Score"] = groupedby_rearange_reduced_final_merger_for_per_school_size_df["math_score"] / groupedby_rearange_reduced_final_merger_for_per_school_size_df["Total Students"]
groupedby_rearange_reduced_final_merger_for_per_school_size_df["Average Reading Score"] = groupedby_rearange_reduced_final_merger_for_per_school_size_df["reading_score"] / groupedby_rearange_reduced_final_merger_for_per_school_size_df["Total Students"]
groupedby_rearange_reduced_final_merger_for_per_school_size_df["% Passing Math"] = (groupedby_rearange_reduced_final_merger_for_per_school_size_df ["Total Passing Math"] / groupedby_rearange_reduced_final_merger_for_per_school_size_df["Total Students"]) * 100
groupedby_rearange_reduced_final_merger_for_per_school_size_df["% Passing Reading"] = (groupedby_rearange_reduced_final_merger_for_per_school_size_df ["Total Passing Reading"] / groupedby_rearange_reduced_final_merger_for_per_school_size_df["Total Students"]) * 100
groupedby_rearange_reduced_final_merger_for_per_school_size_df["% Overall Passing Rate"] = (groupedby_rearange_reduced_final_merger_for_per_school_size_df ["% Passing Math"] + groupedby_rearange_reduced_final_merger_for_per_school_size_df ["% Passing Reading"]) / 2
reduced_groupedby_rearange_reduced_final_merger_for_per_school_size_df = groupedby_rearange_reduced_final_merger_for_per_school_size_df[["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing Rate"]]
reduced_groupedby_rearange_reduced_final_merger_for_per_school_size_df


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small(<1000),83.828654,83.974082,91.360691,92.368611,91.864651
Medium(1000-3000),80.450902,82.626481,78.660484,86.609995,82.635240
Large(3000-5000),77.070764,80.928365,64.335093,78.417070,71.376082


In [47]:
reduced_final_merger_for_per_school_type_df = final_merger_for_per_student_spending_df[['School Type','Total Students','Per Student Budget','Total School Budget', 'reading_score', 'math_score','Total Passing Reading', 'Total Passing Math']]
#reduced_final_merger_for_per_school_type_df

In [48]:
groupedby_reduced_final_merger_for_per_school_type_df = pd.DataFrame(rearange_reduced_final_merger_for_per_school_size_df.groupby("School Type")['Per Student Budget','Total Students', 'Total School Budget', 'reading_score', 'math_score','Total Passing Reading', 'Total Passing Math'].sum())
groupedby_reduced_final_merger_for_per_school_type_df["Average Math Score"] = groupedby_reduced_final_merger_for_per_school_type_df["math_score"] / groupedby_reduced_final_merger_for_per_school_type_df["Total Students"]
groupedby_reduced_final_merger_for_per_school_type_df["Average Reading Score"] = groupedby_reduced_final_merger_for_per_school_type_df["reading_score"] / groupedby_reduced_final_merger_for_per_school_type_df["Total Students"]
groupedby_reduced_final_merger_for_per_school_type_df["% Passing Math"] = (groupedby_reduced_final_merger_for_per_school_type_df ["Total Passing Math"] / groupedby_reduced_final_merger_for_per_school_type_df["Total Students"]) * 100
groupedby_reduced_final_merger_for_per_school_type_df["% Passing Reading"] = (groupedby_reduced_final_merger_for_per_school_type_df ["Total Passing Reading"] / groupedby_reduced_final_merger_for_per_school_type_df["Total Students"]) * 100
groupedby_reduced_final_merger_for_per_school_type_df["% Overall Passing Rate"] = (groupedby_reduced_final_merger_for_per_school_type_df ["% Passing Math"] + groupedby_reduced_final_merger_for_per_school_type_df ["% Passing Reading"]) / 2
reduced_groupedby_reduced_final_merger_for_per_school_type_df = groupedby_reduced_final_merger_for_per_school_type_df[["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing Rate"]]
reduced_groupedby_reduced_final_merger_for_per_school_type_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.406183,83.902821,90.282106,93.152370,91.717238
District,76.987026,80.962485,64.305308,78.369662,71.337485
